In [1]:
import tensorflow as tf
import numpy as np

In [39]:
class WidthConstraint(tf.keras.constraints.Constraint):

    def __init__(self, width=None):
        """
        width: The number of neurons in this layer which should be ignored. They will be set to zero.
        """
        self.width = width

    def __call__(self, w):
        mask = np.zeros(shape=w.shape[-1])
        mask[:self.width] = 1.0
        return w * mask

In [ ]:
class FreezeConstraint(tf.keras.constraints.Constraint):

    def __init__(self, weights=None):
        """
        weights: A weight matrix from a Dense layer of smaller or equal size. If the weight matrix is smaller than necessary for this layer, it will apply the weight constraint only to the nodes of lowest index.
        """
        self.weights = weights

    def __call__(self, w):
        zs = np.zeros(w.shape)
        zs[self.slice] = self.weights
        os = np.ones(w.shape)
        os[self.slice] = 0
        return w * os + zs

In [ ]:
input = tf.keras.Input(100)
layer1 = tf.keras.layers.Dense(100, kernel_constraint=FreezeConstraint())


1) What is the most efficient way to freeze certain neurons in a dense layer in Keras?
    - So they don't contribute at all (zeroed out) (layer which drops them out, multiplies output by 0)
    - So they are frozen (contributes to results but does not change under training) (split layer into trainable and non-trainable portions and use trainable flag. Implement regularizer constraint)

2) Is it possible to specify any neural network based on a dense layer with constraints?

In [16]:
def PartialDense(input, trainable=1, fixed=1, trainable_kwargs={}, fixed_kwargs={}):
    x1 = tf.keras.layers.Dense(trainable, **trainable_kwargs)(input)
    x2 = tf.keras.layers.Dense(fixed, trainable=False, **fixed_kwargs)(input)
    return tf.keras.layers.Concatenate()([x1, x2])

input = tf.keras.Input(100)
layer1 = PartialDense(input, trainable=10, fixed=90)
layer2 = PartialDense(layer1, trainable=10, fixed=90)
concat_model = tf.keras.Model(inputs=input, outputs=layer2)
concat_model.build(input_shape=[100])
concat_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 10)           1010        input_12[0][0]                   
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 90)           9090        input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_21 (Concatenate)    (None, 100)          0           dense_42[0][0]                   
                                                                 dense_43[0][0]             

In [35]:
class PartialDenseLayer(tf.keras.layers.Layer):

    def __init__(self, trainable=1, fixed=1, trainable_kwargs={}, fixed_kwargs={}):
        super().__init__()
        self.fixed = tf.keras.layers.Dense(fixed, trainable=False, **fixed_kwargs)
        self.trainable = tf.keras.layers.Dense(trainable, **trainable_kwargs)

    def call(self, inputs):
        x1 = self.fixed(inputs)
        x2 = self.trainable(inputs)
        return tf.keras.layers.Concatenate()([x1,x2])
        
model = tf.keras.Sequential([
    PartialDenseLayer(trainable=10, fixed=90),
    PartialDenseLayer(trainable=10, fixed=90)
])


RecursionError: maximum recursion depth exceeded